In [ ]:
pip install scapy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 19.7 MB/s eta 0:00:00


In [ ]:
from scapy.all import sniff, conf
import csv
import threading
from datetime import datetime
import queue
import time
from tensorflow.keras.models import load_model
import joblib
import numpy as np
import socket
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

# CONFIG

conf.l3socket = conf.L3socket  # layer-3 sniffing (IP-based)

CSV_FILE = 'netlog.csv'
THRESHOLD = 0.01          # anomaly threshold (adjust based on validation)***

# load pretrained model and scaler
autoencoder = load_model("autoencoder_model.h5")
scaler = joblib.load("scaler.pkl")  # Make sure it matches your training

# init labelEncoder for categorical data
label_encoder = LabelEncoder()

# init scaler for normalization
min_max_scaler = MinMaxScaler()

packet_queue = queue.Queue()  # thread-safe packet queue

# CSV INIT

def init_csv():
    try:
        with open(CSV_FILE, mode='x', newline='') as file:
            writer = csv.writer(file)
            writer.writerow([
                "src_ip", "dst_ip", "protocol", "length",
                "src_port", "dst_port", "flags", "ttl", "payload_size", "protocol_name"
            ])
    except FileExistsError:

# CSV WRITER THREAD

def write_to_csv():
    while True:
        packet_data = packet_queue.get()
        if packet_data is None:
            break
        try:
            with open(CSV_FILE, mode='a', newline='') as file:
                writer = csv.writer(file)
                writer.writerow(packet_data)
        except Exception as e:
            print(f"Error writing packet to CSV: {e}")
        packet_queue.task_done()

# FEATURE EXTRACTION

def ip_to_int(ip):
    """Convert IP address to integer."""
    return int.from_bytes(socket.inet_aton(ip), byteorder='big')

def protocol_to_int(protocol_name):
    """Convert protocol name to integer (can use mapping for ICMP, TCP, UDP)."""
    protocol_mapping = {"ICMP": 1, "TCP": 6, "UDP": 17}
    return protocol_mapping.get(protocol_name, 0)  # default to 0 for unknown protocols

def extract_features(packet_data):
    try:
        # convert IPs to integers using labelEncoder
        src_ip = label_encoder.fit_transform([packet_data[0]])[0]  # Encoding src_ip
        dst_ip = label_encoder.fit_transform([packet_data[1]])[0]  # Encoding dst_ip

        proto = packet_data[2]
        length = packet_data[3]
        src_port = packet_data[4] or 0
        dst_port = packet_data[5] or 0
        flags = packet_data[6] or 'NaN'  # default to 'NaN' if None
        ttl = packet_data[7]
        payload_size = packet_data[8] or 0
        protocol_name = packet_data[9]

        # convert protocol_name to integer (ICMP, TCP, UDP)
        protocol_int = protocol_to_int(protocol_name)

        # map the flags to numeric values***
        flags_map = {'NaN': 0, 'A': 1, 'PA': 2, 'FA': 3, 'S': 4, 'SA': 5,
                     'RA': 6, 'R': 7, 'FPA': 8}
        flags_numeric = flags_map.get(flags, 0)  # Default to 0 for unknown flags

        # create feature vector
        raw_vector = np.array([
            src_ip, dst_ip, proto, length,
            src_port, dst_port, flags_numeric, ttl, payload_size, protocol_int
        ], dtype=float).reshape(1, -1)

        # normalize numerical features
        return min_max_scaler.fit_transform(raw_vector)
    except Exception as e:
        print(f"Feature extraction error: {e}")
        return None

# PACKET SNIFFER CALLBACK

def packet_callback(packet):
    try:
        if packet.haslayer('IP'):
            src_ip = packet['IP'].src
            dst_ip = packet['IP'].dst
            proto = packet.proto
            length = len(packet)
            ttl = packet['IP'].ttl

            # ident for protocol name
            protocol_name = {1: 'ICMP', 6: 'TCP', 17: 'UDP'}.get(proto, 'Other')

            # default values
            src_port = dst_port = flags = payload_size = None

            if packet.haslayer('TCP'):
                src_port = packet['TCP'].sport
                dst_port = packet['TCP'].dport
                flags = packet['TCP'].flags
                payload_size = len(packet['TCP'].payload)
            elif packet.haslayer('UDP'):
                src_port = packet['UDP'].sport
                dst_port = packet['UDP'].dport
                payload_size = len(packet['UDP'].payload)

            # harvest data for feature extraction
            packet_data = [
                src_ip, dst_ip, proto, length,
                src_port, dst_port, flags, ttl, payload_size, protocol_name
            ]

            # anomaly detection***
            features = extract_features(packet_data)
            if features is not None:
                reconstructed = autoencoder.predict(features)
                error = np.mean(np.square(features - reconstructed))

                if error > THRESHOLD:
                    print(f"🚨 Anomaly Detected! [Error={error:.5f}] {src_ip} -> {dst_ip} | {protocol_name}")

            # queue packet data for logging
            packet_queue.put(packet_data)

    except Exception as e:
        print(f"Error processing packet: {e}")

# SNIFFER CONTROL

def start_sniffing():
    print("Sniffing started... Press Ctrl+C to stop.")
    sniff(prn=packet_callback, store=False)

# MAIN EXPERIMENT

if __name__ == "__main__":
    init_csv()

    # spin CSV writer thread
    write_thread = threading.Thread(target=write_to_csv, daemon=True)
    write_thread.start()

    try:
        start_sniffing()
    except KeyboardInterrupt:
        print("Stopping sniffing...")
        packet_queue.put(None)
        write_thread.join()
        print("IDS stopped.")


Sniffing started... Press Ctrl+C to stop.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step
🚨 Anomaly Detected! [Error=0.46567] 172.28.0.1 -> 172.28.0.12 | TCP
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
🚨 Anomaly Detected! [Error=0.46567] 172.28.0.12 -> 172.28.0.1 | TCP
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
🚨 Anomaly Detected! [Error=0.46567] 172.28.0.12 -> 172.28.0.1 | TCP
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
🚨 Anomaly Detected! [Error=0.46567] 172.28.0.1 -> 172.28.0.12 | TCP
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
🚨 Anomaly Detected! [Error=0.46567] 172.28.0.12 -> 172.28.0.1 | TCP
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
🚨 Anomaly Detected! [Error=0.46567] 172.28.0.1 -> 172.28.0.12 | TCP
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
🚨 Anomaly Detected! [Error=0.46567] 172.28.0.12 -> 172.28.0.1 | TCP
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
🚨 Anomaly Detected! [Error=0.46567] 172.28.0.1 -> 172.28.0.12 | TCP
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
🚨 Anomaly Detected! [Error=0.46567] 172.28.0.12 -> 172.28.0.1 | TCP
1/1 